In [2]:
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score
import numpy as np
import json
from sklearn.naive_bayes import BernoulliNB
from sklearn.naive_bayes import MultinomialNB


def calcular_media_equipo(equipo):
    media = {}
    for campeon in equipo:
        for key, value in campeon.items():
            if key != "ID" and key != "Key" and key != "Name":
                if key not in media:
                    media[key] = value
                else:
                    media[key] += value
    num_campeones = len(equipo)
    for key in media:
        media[key] /= num_campeones
    return list(media.values())

def cargar_datos_desde_json(ruta_archivo):
    X = []
    y = []
    with open(ruta_archivo, 'r') as file:
        data = json.load(file)
    for partida in data:
        resultado = partida["result"]
        for equipo in [partida["blue_team"], partida["red_team"]]:
            X.append(calcular_media_equipo(equipo))
            y.append(resultado)
    return np.array(X), np.array(y)

# Cargar datos desde el archivo JSON
X, y = cargar_datos_desde_json("partidasConCaracteristicas.json")

# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Inicializar el clasificador Naive Bayes
naive_bayes = GaussianNB()

# Entrenar el modelo
naive_bayes.fit(X_train, y_train)

# Predecir en el conjunto de prueba
y_pred = naive_bayes.predict(X_test)

# Calcular la precisión del modelo
precision = accuracy_score(y_test, y_pred)
print("Precisión del modelo Naive Bayes:", precision)



# Inicializar el clasificador Naive Bayes Multinomial
naive_bayes_multinomial = MultinomialNB()

# Entrenar el modelo
naive_bayes_multinomial.fit(X_train, y_train)

# Predecir en el conjunto de prueba
y_pred_multinomial = naive_bayes_multinomial.predict(X_test)

# Calcular la precisión del modelo
precision_multinomial = accuracy_score(y_test, y_pred_multinomial)
print("Precisión del modelo Naive Bayes Multinomial:", precision_multinomial)



# Inicializar el clasificador Naive Bayes Bernoulli
naive_bayes_bernoulli = BernoulliNB()

# Entrenar el modelo
naive_bayes_bernoulli.fit(X_train, y_train)

# Predecir en el conjunto de prueba
y_pred_bernoulli = naive_bayes_bernoulli.predict(X_test)

# Calcular la precisión del modelo
precision_bernoulli = accuracy_score(y_test, y_pred_bernoulli)
print("Precisión del modelo Naive Bayes Bernoulli:", precision_bernoulli)




Precisión del modelo Naive Bayes: 0.5368153790351832
Precisión del modelo Naive Bayes Multinomial: 0.5369967355821545
Precisión del modelo Naive Bayes Bernoulli: 0.5369967355821545


In [5]:
import json

# Cargar y visualizar el contenido de "caracteristicas_campeones.json"
with open("caracteristicas_campeones.json", "r") as file:
    champion_features = json.load(file)

# Cargar y visualizar el contenido de "arraysPartidasResultado.json"
with open("arraysPartidasResultado.json", "r") as file:
    match_results = json.load(file)

    keys = ['Difficulty', 'HP', 'Armor', 'HPRegen', 'AttackDamage', 'AttackSpeed']
neutral_stats = {key: np.mean([champ[key] for champ in champion_features]) for key in keys}
neutral_stats['Name'] = 'Unknown'
neutral_stats['Key'] = -1

# Función de transformación actualizada para manejar campeones no encontrados
def transform_match_updated(match, champion_dict, neutral_stats):
    result = match[0]
    # Utilizar estadísticas neutrales si el campeón no está en el diccionario
    blue_team = [champion_dict.get(champ_id, neutral_stats) for champ_id in match[1:6]]
    red_team = [champion_dict.get(champ_id, neutral_stats) for champ_id in match[6:11]]
    return result, blue_team, red_team


all_matches_transformed = [transform_match_updated(match, champion_dict, neutral_stats) for match in match_results]

# Convertir las características de cada campeón en cada equipo en un solo vector para cada partida
def flatten_match_data(match_data):
    result, blue_team, red_team = match_data
    # Aplanar las características de los campeones en un solo vector
    blue_features = [value for champ in blue_team for key, value in champ.items() if isinstance(value, (int, float))]
    red_features = [value for champ in red_team for key, value in champ.items() if isinstance(value, (int, float))]
    features = blue_features + red_features
    return result, features

# Aplanar los datos para todos los partidos
flattened_data = [flatten_match_data(match) for match in all_matches_transformed]

# Extraer las características y los resultados
X = np.array([features for _, features in flattened_data])
y = np.array([result for result, _ in flattened_data])

# Normalizar las características
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Muestra del resultado final procesado y normalizado
X_scaled.shape, y.shape


((13783, 70), (13783,))

In [6]:
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# Inicializar el clasificador Naive Bayes
nb = GaussianNB()

# Entrenar el modelo Naive Bayes
nb.fit(X_train, y_train)

# Predecir los resultados para el conjunto de prueba
y_pred_nb = nb.predict(X_test)

# Calcular y mostrar métricas de rendimiento
accuracy_nb = accuracy_score(y_test, y_pred_nb)
conf_matrix_nb = confusion_matrix(y_test, y_pred_nb)
report_nb = classification_report(y_test, y_pred_nb, target_names=["Perder", "Ganar"])

# Presentación de los resultados
print("Precisión del modelo:", accuracy_nb)
print("Matriz de confusión:\n", conf_matrix_nb)
print("Reporte de clasificación:\n", report_nb)


Precisión del modelo: 0.544069640914037
Matriz de confusión:
 [[ 225 1019]
 [ 238 1275]]
Reporte de clasificación:
               precision    recall  f1-score   support

      Perder       0.49      0.18      0.26      1244
       Ganar       0.56      0.84      0.67      1513

    accuracy                           0.54      2757
   macro avg       0.52      0.51      0.47      2757
weighted avg       0.52      0.54      0.49      2757


In [7]:

from sklearn.naive_bayes import BernoulliNB
from sklearn.preprocessing import Binarizer

# Binarizar las características
binarizer = Binarizer()
X_binarized = binarizer.fit_transform(X_scaled)

# Dividir los datos en conjuntos de entrenamiento y prueba
X_train_bin, X_test_bin, y_train, y_test = train_test_split(X_binarized, y, test_size=0.2, random_state=42)

# Inicializar y entrenar el clasificador
bnb = BernoulliNB()
bnb.fit(X_train_bin, y_train)

# Realizar predicciones
y_pred_bnb = bnb.predict(X_test_bin)

# Evaluar el modelo
print("Accuracy:", accuracy_score(y_test, y_pred_bnb))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred_bnb))
print("Classification Report:\n", classification_report(y_test, y_pred_bnb))


Accuracy: 0.5505984766050055
Confusion Matrix:
 [[529 715]
 [524 989]]
Classification Report:
               precision    recall  f1-score   support

           0       0.50      0.43      0.46      1244
           1       0.58      0.65      0.61      1513

    accuracy                           0.55      2757
   macro avg       0.54      0.54      0.54      2757
weighted avg       0.55      0.55      0.55      2757


In [8]:
from sklearn.naive_bayes import BernoulliNB
from sklearn.preprocessing import Binarizer

# Binarizar las características para Bernoulli Naive Bayes
binarizer = Binarizer()
X_binarized = binarizer.fit_transform(X_scaled)

# Dividir los datos binarizados en conjuntos de entrenamiento y prueba
X_train_bin, X_test_bin, y_train, y_test = train_test_split(X_binarized, y, test_size=0.2, random_state=42)

# Inicializar el clasificador Bernoulli Naive Bayes
bnb = BernoulliNB()

# Entrenar el modelo Bernoulli Naive Bayes
bnb.fit(X_train_bin, y_train)

# Predecir los resultados para el conjunto de prueba
y_pred_bnb = bnb.predict(X_test_bin)

# Calcular y mostrar métricas de rendimiento para Bernoulli Naive Bayes
accuracy_bnb = accuracy_score(y_test, y_pred_bnb)
conf_matrix_bnb = confusion_matrix(y_test, y_pred_bnb)
report_bnb = classification_report(y_test, y_pred_bnb, target_names=["Perder", "Ganar"])

accuracy_bnb, conf_matrix_bnb, report_bnb


(0.5505984766050055,
 array([[529, 715],
        [524, 989]], dtype=int64),
 '              precision    recall  f1-score   support\n\n      Perder       0.50      0.43      0.46      1244\n       Ganar       0.58      0.65      0.61      1513\n\n    accuracy                           0.55      2757\n   macro avg       0.54      0.54      0.54      2757\nweighted avg       0.55      0.55      0.55      2757\n')